In [1]:
import torch
from torch.utils.data import DataLoader
import torch.utils.model_zoo as model_zoo
import torchvision.models as models

from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model, assemble_model_with_classifier, load_model_merged
from model.utils import freeze_layers
from model.densenet import *
from trainer.trainer import get_trainer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [2]:
# configuration
config = {
    'train_batch_size': 512, 'val_batch_size': 512,
    'arch': 'densenet161', 'pretrained': True,
    'optimizer': 'Adam', 'learning_rate': 1e-3, 'momentum': 0.9, 'decay_lr_freq': 2e4, 'weight_decay': 1e-4,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 2e4, 'save_freq': 1e3,
    'best_val_prec1': 0
}

In [ ]:
# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                      images_csv="train_images.csv",
                                      bson_file_path="/mnt/data/cdiscount/train.bson",
                                      with_label=True,
                                      resize=160)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                    images_csv="val_images.csv",
                                    bson_file_path="/mnt/data/cdiscount/train.bson",
                                    with_label=True,
                                    resize=160)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=1)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
pretrained_model = models.__dict__[config['arch']](pretrained=config['pretrained'])

# model = torch.nn.Sequential(*list(model.children())[:-1])
# model.add_module('classifier', torch.nn.Linear(in_features=2048, out_features=5270))
# model = torch.nn.DataParallel(model).cuda()
# print(model)

#model = load_model_merged(name=config['arch'], num_classes=5270)
model = densenet161(num_classes=5270)

pretrained_state = pretrained_model.state_dict()
model_state = model.state_dict()

for name, state in pretrained_state.items():
    if not name.startswith('classifier'):
        model_state[name].copy_(state)

# freeze_layers(model, 6)
n_layers = 9
i = 0
for child in model.children():
    for subchild in child.children():
        if i >= n_layers:
            break
        print(i, "freezing", subchild)
        for param in subchild.parameters():
            param.requires_grad = False
        i += 1
        

# classifier_layer = [
#     torch.nn.Linear(in_features=2208, out_features=5270)
# ]

# # classifier_layer = [
# #     torch.nn.Linear(in_features=2048, out_features=5270),
# # ]

# classifier = torch.nn.Sequential(*classifier_layer)

# model = assemble_model_with_classifier(model, -1, torch.nn.Linear(in_features=2208, out_features=5270))
model = torch.nn.DataParallel(model).cuda()
print(model)

getting dataset...
getting data loader...
=> using pre-trained model 'densenet161'
0 freezing Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
1 freezing BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
2 freezing ReLU (inplace)
3 freezing MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
4 freezing _DenseBlock (
  (denselayer1): _DenseLayer (
    (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (relu.2): ReLU (inplace)
    (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (denselayer2): _DenseLayer (
    (norm.1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm.2): BatchNorm

In [ ]:
# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

start training
Epoch: [0][0/19336]	Time 97.360 (97.360)	Data 2.998 (2.998)	Loss 8.6644 (8.6644)	Prec@1 0.000 (0.000)	Prec@5 0.195 (0.195)
Epoch: [0][10/19336]	Time 6.075 (14.363)	Data 0.012 (0.285)	Loss 7.1972 (7.6138)	Prec@1 9.961 (7.120)	Prec@5 18.945 (14.062)
Epoch: [0][20/19336]	Time 6.113 (10.426)	Data 0.013 (0.156)	Loss 6.2440 (7.0476)	Prec@1 15.625 (10.128)	Prec@5 27.539 (19.996)
Epoch: [0][30/19336]	Time 6.103 (9.033)	Data 0.013 (0.110)	Loss 5.8004 (6.6869)	Prec@1 15.820 (11.864)	Prec@5 30.273 (23.173)
Epoch: [0][40/19336]	Time 6.114 (8.318)	Data 0.014 (0.087)	Loss 5.4047 (6.4069)	Prec@1 19.531 (13.686)	Prec@5 34.570 (25.953)
Epoch: [0][50/19336]	Time 6.124 (7.882)	Data 0.016 (0.073)	Loss 5.0729 (6.1727)	Prec@1 23.047 (15.219)	Prec@5 42.188 (28.389)
Epoch: [0][60/19336]	Time 6.089 (7.591)	Data 0.013 (0.064)	Loss 5.1486 (6.0053)	Prec@1 23.047 (16.531)	Prec@5 37.891 (30.197)
Epoch: [0][70/19336]	Time 6.123 (7.383)	Data 0.020 (0.057)	Loss 4.7306 (5.8571)	Prec@1 25.000 (17.564)	Pre

Epoch: [0][650/19336]	Time 6.095 (6.243)	Data 0.019 (0.021)	Loss 3.7374 (4.2490)	Prec@1 31.445 (30.539)	Prec@5 56.055 (48.919)
Epoch: [0][660/19336]	Time 6.126 (6.241)	Data 0.052 (0.021)	Loss 3.6472 (4.2393)	Prec@1 38.477 (30.635)	Prec@5 56.641 (49.032)
Epoch: [0][670/19336]	Time 6.118 (6.239)	Data 0.015 (0.021)	Loss 3.9780 (4.2313)	Prec@1 32.227 (30.709)	Prec@5 51.172 (49.124)
Epoch: [0][680/19336]	Time 6.109 (6.237)	Data 0.019 (0.021)	Loss 3.6463 (4.2227)	Prec@1 35.156 (30.786)	Prec@5 57.031 (49.226)
Epoch: [0][690/19336]	Time 6.106 (6.235)	Data 0.019 (0.021)	Loss 3.3565 (4.2139)	Prec@1 37.891 (30.855)	Prec@5 58.398 (49.319)
Epoch: [0][700/19336]	Time 6.097 (6.233)	Data 0.015 (0.021)	Loss 3.5193 (4.2048)	Prec@1 36.133 (30.941)	Prec@5 55.273 (49.428)
Epoch: [0][710/19336]	Time 6.115 (6.231)	Data 0.015 (0.021)	Loss 3.6187 (4.1965)	Prec@1 35.156 (31.024)	Prec@5 54.883 (49.527)
Epoch: [0][720/19336]	Time 6.081 (6.229)	Data 0.015 (0.021)	Loss 3.4950 (4.1881)	Prec@1 39.258 (31.116)	Prec@5 

Epoch: [0][1300/19336]	Time 6.128 (6.180)	Data 0.022 (0.022)	Loss 3.3229 (3.8722)	Prec@1 40.234 (34.300)	Prec@5 59.375 (53.268)
Epoch: [0][1310/19336]	Time 6.093 (6.180)	Data 0.020 (0.022)	Loss 3.5769 (3.8684)	Prec@1 36.523 (34.337)	Prec@5 56.055 (53.312)
Epoch: [0][1320/19336]	Time 6.097 (6.179)	Data 0.023 (0.022)	Loss 3.2891 (3.8649)	Prec@1 40.625 (34.374)	Prec@5 58.398 (53.354)
Epoch: [0][1330/19336]	Time 6.113 (6.179)	Data 0.024 (0.022)	Loss 3.6493 (3.8616)	Prec@1 34.375 (34.402)	Prec@5 53.516 (53.383)
Epoch: [0][1340/19336]	Time 6.115 (6.179)	Data 0.021 (0.022)	Loss 3.2463 (3.8584)	Prec@1 39.844 (34.430)	Prec@5 59.180 (53.424)
Epoch: [0][1350/19336]	Time 6.125 (6.178)	Data 0.039 (0.022)	Loss 3.5842 (3.8547)	Prec@1 37.109 (34.468)	Prec@5 56.055 (53.467)
Epoch: [0][1360/19336]	Time 6.151 (6.178)	Data 0.027 (0.022)	Loss 3.2471 (3.8512)	Prec@1 41.016 (34.502)	Prec@5 59.375 (53.508)
Epoch: [0][1370/19336]	Time 6.157 (6.177)	Data 0.021 (0.022)	Loss 3.2409 (3.8481)	Prec@1 41.016 (34.534)

Epoch: [0][1950/19336]	Time 6.123 (6.162)	Data 0.031 (0.023)	Loss 3.4217 (3.6925)	Prec@1 38.477 (36.182)	Prec@5 58.789 (55.405)
Epoch: [0][1960/19336]	Time 6.130 (6.162)	Data 0.031 (0.024)	Loss 3.3537 (3.6904)	Prec@1 41.797 (36.209)	Prec@5 59.570 (55.433)
Epoch: [0][1970/19336]	Time 6.131 (6.162)	Data 0.038 (0.024)	Loss 3.4758 (3.6886)	Prec@1 36.523 (36.223)	Prec@5 57.617 (55.451)
Epoch: [0][1980/19336]	Time 6.110 (6.162)	Data 0.026 (0.024)	Loss 3.2077 (3.6862)	Prec@1 42.188 (36.246)	Prec@5 62.891 (55.481)
Epoch: [0][1990/19336]	Time 6.106 (6.161)	Data 0.035 (0.024)	Loss 3.2050 (3.6839)	Prec@1 42.969 (36.266)	Prec@5 59.961 (55.506)
Epoch: [0][2000/19336]	Time 6.121 (6.161)	Data 0.028 (0.024)	Loss 3.1017 (3.6816)	Prec@1 42.188 (36.293)	Prec@5 62.109 (55.535)
Epoch: [0][2010/19336]	Time 6.144 (6.161)	Data 0.025 (0.024)	Loss 3.3886 (3.6796)	Prec@1 40.625 (36.314)	Prec@5 58.398 (55.556)
Epoch: [0][2020/19336]	Time 6.110 (6.161)	Data 0.024 (0.024)	Loss 3.4032 (3.6776)	Prec@1 40.039 (36.338)

Epoch: [0][2600/19336]	Time 6.137 (6.154)	Data 0.031 (0.026)	Loss 3.0972 (3.5757)	Prec@1 42.188 (37.473)	Prec@5 65.430 (56.818)
Epoch: [0][2610/19336]	Time 6.109 (6.154)	Data 0.038 (0.026)	Loss 3.2741 (3.5742)	Prec@1 41.211 (37.490)	Prec@5 60.156 (56.836)
Epoch: [0][2620/19336]	Time 6.123 (6.154)	Data 0.029 (0.026)	Loss 3.2421 (3.5730)	Prec@1 40.625 (37.502)	Prec@5 63.086 (56.851)
Epoch: [0][2630/19336]	Time 6.138 (6.154)	Data 0.032 (0.026)	Loss 2.9408 (3.5716)	Prec@1 44.922 (37.520)	Prec@5 64.844 (56.869)
Epoch: [0][2640/19336]	Time 6.136 (6.153)	Data 0.041 (0.026)	Loss 3.0869 (3.5702)	Prec@1 44.336 (37.537)	Prec@5 62.500 (56.887)
Epoch: [0][2650/19336]	Time 6.135 (6.153)	Data 0.038 (0.026)	Loss 3.3805 (3.5688)	Prec@1 40.234 (37.552)	Prec@5 57.812 (56.904)
Epoch: [0][2660/19336]	Time 6.097 (6.153)	Data 0.032 (0.026)	Loss 3.3009 (3.5675)	Prec@1 41.211 (37.568)	Prec@5 60.352 (56.919)
Epoch: [0][2670/19336]	Time 6.113 (6.153)	Data 0.036 (0.026)	Loss 3.2774 (3.5663)	Prec@1 41.016 (37.582)

...

Epoch: [0][7150/19336]	Time 6.135 (6.136)	Data 0.045 (0.032)	Loss 2.9579 (3.2659)	Prec@1 45.117 (41.006)	Prec@5 66.016 (60.673)
Epoch: [0][7160/19336]	Time 6.113 (6.136)	Data 0.038 (0.032)	Loss 3.1435 (3.2656)	Prec@1 42.773 (41.009)	Prec@5 62.109 (60.676)
Epoch: [0][7170/19336]	Time 6.143 (6.136)	Data 0.045 (0.032)	Loss 2.8959 (3.2653)	Prec@1 48.047 (41.013)	Prec@5 63.672 (60.680)
Epoch: [0][7180/19336]	Time 6.141 (6.136)	Data 0.041 (0.032)	Loss 3.0843 (3.2648)	Prec@1 40.625 (41.018)	Prec@5 63.672 (60.685)
Epoch: [0][7190/19336]	Time 6.123 (6.136)	Data 0.039 (0.032)	Loss 3.1477 (3.2645)	Prec@1 43.359 (41.022)	Prec@5 62.500 (60.690)
Epoch: [0][7200/19336]	Time 6.116 (6.136)	Data 0.042 (0.032)	Loss 2.9381 (3.2641)	Prec@1 46.289 (41.027)	Prec@5 63.477 (60.695)
Epoch: [0][7210/19336]	Time 6.105 (6.136)	Data 0.035 (0.032)	Loss 3.2019 (3.2638)	Prec@1 41.016 (41.030)	Prec@5 61.914 (60.699)
Epoch: [0][7220/19336]	Time 6.115 (6.136)	Data 0.044 (0.032)	Loss 3.2061 (3.2635)	Prec@1 43.164 (41.034)

Epoch: [0][7800/19336]	Time 6.127 (6.135)	Data 0.042 (0.032)	Loss 2.8530 (3.2449)	Prec@1 47.461 (41.261)	Prec@5 65.430 (60.932)
Epoch: [0][7810/19336]	Time 6.127 (6.135)	Data 0.040 (0.032)	Loss 3.2043 (3.2446)	Prec@1 38.672 (41.264)	Prec@5 60.547 (60.936)
Epoch: [0][7820/19336]	Time 6.066 (6.135)	Data 0.041 (0.032)	Loss 2.9168 (3.2444)	Prec@1 46.680 (41.267)	Prec@5 64.844 (60.940)
Epoch: [0][7830/19336]	Time 6.102 (6.135)	Data 0.045 (0.032)	Loss 3.0417 (3.2441)	Prec@1 44.531 (41.271)	Prec@5 64.062 (60.944)
Epoch: [0][7840/19336]	Time 6.107 (6.135)	Data 0.035 (0.032)	Loss 2.9665 (3.2438)	Prec@1 45.117 (41.274)	Prec@5 64.258 (60.946)
Epoch: [0][7850/19336]	Time 6.148 (6.135)	Data 0.037 (0.032)	Loss 2.8501 (3.2435)	Prec@1 46.680 (41.279)	Prec@5 66.211 (60.952)
Epoch: [0][7860/19336]	Time 6.125 (6.134)	Data 0.043 (0.032)	Loss 3.2717 (3.2431)	Prec@1 42.578 (41.282)	Prec@5 62.891 (60.955)
Epoch: [0][7870/19336]	Time 6.109 (6.134)	Data 0.038 (0.032)	Loss 3.0720 (3.2428)	Prec@1 43.359 (41.285)

Epoch: [0][8450/19336]	Time 6.130 (6.134)	Data 0.040 (0.033)	Loss 2.9521 (3.2264)	Prec@1 45.898 (41.479)	Prec@5 66.016 (61.165)
Epoch: [0][8460/19336]	Time 6.109 (6.134)	Data 0.037 (0.033)	Loss 2.6834 (3.2262)	Prec@1 47.266 (41.481)	Prec@5 67.578 (61.168)
Epoch: [0][8470/19336]	Time 6.073 (6.134)	Data 0.042 (0.033)	Loss 3.2190 (3.2260)	Prec@1 43.750 (41.483)	Prec@5 61.523 (61.171)
Epoch: [0][8480/19336]	Time 6.135 (6.134)	Data 0.046 (0.033)	Loss 3.0057 (3.2257)	Prec@1 45.312 (41.487)	Prec@5 64.844 (61.174)
Epoch: [0][8490/19336]	Time 6.126 (6.134)	Data 0.050 (0.033)	Loss 3.1520 (3.2255)	Prec@1 41.602 (41.488)	Prec@5 63.086 (61.175)
Epoch: [0][8500/19336]	Time 6.138 (6.134)	Data 0.050 (0.033)	Loss 3.0634 (3.2255)	Prec@1 45.703 (41.490)	Prec@5 65.625 (61.177)
Epoch: [0][8510/19336]	Time 6.160 (6.134)	Data 0.039 (0.033)	Loss 3.0261 (3.2253)	Prec@1 45.312 (41.492)	Prec@5 63.867 (61.179)
Epoch: [0][8520/19336]	Time 6.142 (6.133)	Data 0.042 (0.033)	Loss 2.8712 (3.2249)	Prec@1 46.875 (41.497)

Epoch: [0][9100/19336]	Time 6.134 (6.133)	Data 0.040 (0.034)	Loss 3.1555 (3.2098)	Prec@1 45.508 (41.666)	Prec@5 61.914 (61.376)
Epoch: [0][9110/19336]	Time 6.098 (6.133)	Data 0.045 (0.034)	Loss 2.9649 (3.2096)	Prec@1 43.945 (41.670)	Prec@5 64.453 (61.379)
Epoch: [0][9120/19336]	Time 6.104 (6.133)	Data 0.039 (0.034)	Loss 3.0235 (3.2094)	Prec@1 43.164 (41.673)	Prec@5 63.672 (61.383)
Epoch: [0][9130/19336]	Time 6.127 (6.133)	Data 0.044 (0.034)	Loss 3.1484 (3.2091)	Prec@1 41.211 (41.676)	Prec@5 62.305 (61.387)
Epoch: [0][9140/19336]	Time 6.124 (6.133)	Data 0.047 (0.034)	Loss 3.0527 (3.2089)	Prec@1 42.383 (41.679)	Prec@5 61.914 (61.389)
Epoch: [0][9150/19336]	Time 6.130 (6.133)	Data 0.041 (0.034)	Loss 2.9817 (3.2086)	Prec@1 44.922 (41.682)	Prec@5 66.016 (61.392)
Epoch: [0][9160/19336]	Time 6.158 (6.133)	Data 0.053 (0.034)	Loss 3.1546 (3.2084)	Prec@1 41.992 (41.685)	Prec@5 62.891 (61.396)
Epoch: [0][9170/19336]	Time 6.110 (6.133)	Data 0.050 (0.034)	Loss 3.0556 (3.2082)	Prec@1 45.508 (41.688)

Epoch: [0][9750/19336]	Time 6.152 (6.133)	Data 0.054 (0.036)	Loss 2.7047 (3.1944)	Prec@1 50.195 (41.860)	Prec@5 68.945 (61.578)
Epoch: [0][9760/19336]	Time 6.132 (6.133)	Data 0.041 (0.036)	Loss 2.8782 (3.1941)	Prec@1 44.336 (41.863)	Prec@5 65.039 (61.581)
Epoch: [0][9770/19336]	Time 6.093 (6.133)	Data 0.047 (0.036)	Loss 3.3107 (3.1939)	Prec@1 40.234 (41.866)	Prec@5 58.594 (61.583)
Epoch: [0][9780/19336]	Time 6.107 (6.133)	Data 0.041 (0.036)	Loss 2.9199 (3.1937)	Prec@1 44.727 (41.869)	Prec@5 65.234 (61.586)
Epoch: [0][9790/19336]	Time 6.120 (6.133)	Data 0.046 (0.036)	Loss 3.1759 (3.1934)	Prec@1 42.969 (41.872)	Prec@5 61.719 (61.589)
Epoch: [0][9800/19336]	Time 6.101 (6.132)	Data 0.048 (0.036)	Loss 2.9525 (3.1932)	Prec@1 43.164 (41.875)	Prec@5 63.867 (61.591)
Epoch: [0][9810/19336]	Time 6.079 (6.132)	Data 0.052 (0.036)	Loss 3.1312 (3.1930)	Prec@1 42.188 (41.876)	Prec@5 63.867 (61.594)
Epoch: [0][9820/19336]	Time 6.101 (6.132)	Data 0.041 (0.036)	Loss 3.0398 (3.1928)	Prec@1 41.992 (41.878)